In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings


In [24]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [25]:
len(final_documents)

316

In [26]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

In [27]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568570e-02 -1.19099393e-02 -3.37892659e-02  2.94559225e-02
  5.19159809e-02  5.73839694e-02 -4.10017669e-02  2.74268202e-02
 -1.05128214e-01 -1.58055983e-02  7.94858634e-02  5.64318448e-02
 -1.31765399e-02 -3.41544002e-02  5.81603777e-03  4.72547822e-02
 -1.30746784e-02  3.12992721e-03 -3.44225913e-02  3.08406502e-02
 -4.09086607e-02  3.52738090e-02 -2.43761223e-02 -4.35831621e-02
  2.41503697e-02  1.31986560e-02 -4.84450301e-03  1.92347504e-02
 -5.43912910e-02 -1.42735064e-01  5.15528582e-03  2.93115824e-02
 -5.60810864e-02 -8.53534415e-03  3.14141326e-02  2.76736412e-02
 -2.06188057e-02  8.24231580e-02  4.15425599e-02  5.79655133e-02
 -3.71587165e-02  6.26160717e-03 -2.41390113e-02 -5.61794452e-03
 -2.51715109e-02  5.04971296e-03 -2.52801068e-02 -2.91943084e-03
 -8.24046228e-03 -5.69604449e-02  2.30822824e-02 -5.54217445e-03
  5.11555672e-02  6.09937720e-02  6.49766251e-02 -5.38513809e-02
  2.19109580e-02 -2.54194364e-02 -4.49222922e-02  4.22459133e-02
  4.75252345e-02  7.23218

In [28]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [29]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [ ]:
from dotenv import load_dotenv
import os
# Load environment variables from the .env file
load_dotenv()

In [ ]:
# Access the Hugging Face API token
huggingface_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    temperature=0.1,
    model_kwargs={"max_length": 500},
    huggingfacehub_api_token=huggingface_api_token
)

query="What is the health insurance coverage?"
llm.invoke(query)